<a href="https://colab.research.google.com/github/vibhuvanjari-Kellogg/MSDS422-Airlines-Project/blob/main/Final_Project_Part_3_Opportunity_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction
Taking the perspective of the CEO of United in August 2015, the goal is to analyze the Skytrax Reviews Dataset to find focus areas for the customer experience.

Link to Part 1 (EDA): https://colab.research.google.com/drive/1e3BaBqnoDy3yb_xvaFqOZJYuhz_YEDMv?usp=sharing

Link to Part 2 (NA Carrier Analysis): https://colab.research.google.com/drive/1_itJ0k2WhjNTh1aKawhtxJn3zaLWCyQV?usp=sharing

Link to Part 3 (Opportunity Analysis): https://colab.research.google.com/drive/1A0AuW30CiUdbjBJzSzLt_p-6sO55sIc_?usp=sharing

### Import modules and data

In [ ]:
# Import modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
import statsmodels.formula.api as smf

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# Figures inline and set visualization style
%matplotlib inline
sns.set()

# To ensure all columns are displayed when calling data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df_filtered_na = pd.read_csv('df_filtered_na.csv')

In [ ]:
df_filtered_na.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3261 entries, 0 to 3260
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   airline_name            3261 non-null   object 
 1   link                    3261 non-null   object 
 2   title                   3261 non-null   object 
 3   author                  3261 non-null   object 
 4   author_country          3255 non-null   object 
 5   date                    3261 non-null   object 
 6   content                 3261 non-null   object 
 7   aircraft                143 non-null    object 
 8   type_traveller          335 non-null    object 
 9   cabin_flown             3058 non-null   object 
 10  route                   329 non-null    object 
 11  overall                 3052 non-null   float64
 12  seat_comfort            3098 non-null   float64
 13  cabin_staff             3099 non-null   float64
 14  food_beverages          3047 non-null   

In [ ]:
df_filtered_na.head()

,airline_name,link,title,author,author_country,date,content,aircraft,type_traveller,cabin_flown,route,overall,seat_comfort,cabin_staff,food_beverages,inflight_entertainment,ground_service,wifi_connectivity,value_money,recommended,year,month,day,100pluscount,airline_type,author_subregion,author_region,origin,destination,layover
0,air-canada,/airline-reviews/air-canada,Air Canada customer review,Anu Gunasekera,Canada,2015-07-30,London to Toronto. Flight delayed 5 hrs and no...,Boeing 767,FamilyLeisure,Economy,London Heathrow to Toronto,3.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,0,2015,7,30,1,na_legacy,Northern America,North America,London Heathrow,Toronto,NaN
1,air-canada,/airline-reviews/air-canada,Air Canada customer review,E R Subra,United States,2015-07-29,We had a confirmed booking to fly from Toronto...,NaN,Couple Leisure,Economy,YYZto EWR,1.0,1.0,1.0,1.0,2.0,1.0,NaN,1.0,0,2015,7,29,1,na_legacy,Northern America,North America,Toronto,Newark,NaN
2,air-canada,/airline-reviews/air-canada,Air Canada customer review,Nariner Rakkar,United Kingdom,2015-07-29,I travelled 27th July from Toronto to LHR. Thi...,Boeing 767-300,FamilyLeisure,Economy,YYZ to LHR,1.0,1.0,4.0,2.0,1.0,2.0,NaN,1.0,0,2015,7,29,1,na_legacy,Northern Europe,Europe,YYZ,LHR,NaN
3,air-canada,/airline-reviews/air-canada,Air Canada customer review,T Eldem,Canada,2015-07-29,Slow check-in in Beijing. Used the Air China B...,Boeing 777 and CRJ,Business,Business Class,PEK to YUL via YYZ,5.0,2.0,2.0,2.0,2.0,2.0,NaN,3.0,1,2015,7,29,1,na_legacy,Northern America,North America,PEK,YUL,YYZ
4,air-canada,/airline-reviews/air-canada,Air Canada customer review,James Crawford,Canada,2015-07-26,Indifferent cabin crew acted like passengers w...,A330,Solo Leisure,Economy,Frankfurt to Montreal,3.0,3.0,2.0,1.0,3.0,3.0,NaN,2.0,0,2015,7,26,1,na_legacy,Northern America,North America,Frankfurt,Montreal,NaN


### Rating analysis

In [ ]:
# value_money by airline_name
df_filtered_na.groupby('airline_name')['value_money'].mean().sort_values(ascending=False)

,value_money
airline_name,
jetblue-airways,3.477477
southwest-airlines,3.398417
alaska-airlines,3.199153
delta-air-lines,2.987124
air-canada,2.718941
american-airlines,2.406863
united-airlines,2.239286


In [ ]:
# Create the pivot table
value_money_table = df_filtered_na.pivot_table(
    index='airline_name',
    columns='cabin_flown',
    values='value_money',
    aggfunc='mean'
)

# Add row-wise average to sort
value_money_table['Average'] = value_money_table.mean(axis=1)
value_money_table = value_money_table.sort_values('Average', ascending=False)
value_money_table = value_money_table.drop(columns='Average')

# Compute average across all airlines
overall_avg = pd.DataFrame(value_money_table.mean(axis=0)).T
overall_avg.index = ['All Airlines']

# Append to the bottom
value_money_table = pd.concat([value_money_table, overall_avg])

# Round for readability
value_money_table = value_money_table.round(2)

# Display
value_money_table


cabin_flown,Business Class,Economy,First Class,Premium Economy
jetblue-airways,5.00,3.73,NaN,NaN
alaska-airlines,5.00,3.24,3.10,NaN
air-canada,3.10,2.60,3.29,4.33
delta-air-lines,3.54,2.87,3.02,2.96
american-airlines,3.04,2.08,3.28,3.00
united-airlines,2.68,2.03,2.54,2.55
southwest-airlines,2.80,3.41,NaN,1.00
All Airlines,3.59,2.85,3.04,2.77


In [ ]:
# Create the pivot table
cabin_staff_table = df_filtered_na.pivot_table(
    index='airline_name',
    columns='cabin_flown',
    values='cabin_staff',
    aggfunc='mean'
)

# Add row-wise average to sort
cabin_staff_table['Average'] = cabin_staff_table.mean(axis=1)
cabin_staff_table = cabin_staff_table.sort_values('Average', ascending=False)
cabin_staff_table = cabin_staff_table.drop(columns='Average')

# Compute average across all airlines
overall_avg = pd.DataFrame(cabin_staff_table.mean(axis=0)).T
overall_avg.index = ['All Airlines']

# Append to the bottom
cabin_staff_table = pd.concat([cabin_staff_table, overall_avg])

# Round for readability
cabin_staff_table = cabin_staff_table.round(2)

# Display
cabin_staff_table


cabin_flown,Business Class,Economy,First Class,Premium Economy
jetblue-airways,5.00,3.82,NaN,NaN
alaska-airlines,5.00,3.73,3.59,NaN
delta-air-lines,4.05,3.21,3.46,3.35
air-canada,3.75,2.70,3.71,3.67
american-airlines,3.16,2.15,3.71,2.67
united-airlines,2.95,2.23,2.89,2.50
southwest-airlines,2.75,3.50,NaN,1.00
All Airlines,3.81,3.05,3.47,2.64


In [ ]:
# Create the pivot table
seat_comfort_table = df_filtered_na.pivot_table(
    index='airline_name',
    columns='cabin_flown',
    values='seat_comfort',
    aggfunc='mean'
)

# Add row-wise average to sort
seat_comfort_table['Average'] = seat_comfort_table.mean(axis=1)
seat_comfort_table = seat_comfort_table.sort_values('Average', ascending=False)
seat_comfort_table = seat_comfort_table.drop(columns='Average')

# Compute average across all airlines
overall_avg = pd.DataFrame(seat_comfort_table.mean(axis=0)).T
overall_avg.index = ['All Airlines']

# Append to the bottom
seat_comfort_table = pd.concat([seat_comfort_table, overall_avg])

# Round for readability
seat_comfort_table = seat_comfort_table.round(2)

# Display
seat_comfort_table


cabin_flown,Business Class,Economy,First Class,Premium Economy
jetblue-airways,5.00,3.86,NaN,NaN
air-canada,3.33,2.66,3.86,3.67
alaska-airlines,3.00,3.31,3.48,NaN
delta-air-lines,3.90,2.96,3.39,2.52
american-airlines,3.38,2.08,3.54,2.44
united-airlines,3.16,2.15,3.02,2.72
southwest-airlines,3.00,3.18,NaN,1.00
All Airlines,3.54,2.89,3.46,2.47


### Origin analysis

In [ ]:
# Count reviews by airline and author_region
region_counts = df_filtered_na.groupby(['airline_name', 'author_region']).size().unstack(fill_value=0)

# Optional: sort rows by total number of reviews
region_counts['Total'] = region_counts.sum(axis=1)
region_counts = region_counts.sort_values('Total', ascending=False).drop(columns='Total')

# Display
region_counts


author_region,Africa,Asia,Europe,North America,Oceania,South America,Unknown
airline_name,,,,,,,
united-airlines,3,45,129,612,42,8,1
american-airlines,3,17,160,399,18,11,4
air-canada,0,19,94,353,23,1,1
delta-air-lines,3,20,88,323,24,8,0
southwest-airlines,2,4,21,326,21,5,0
jetblue-airways,3,9,29,189,5,2,0
alaska-airlines,0,5,13,192,24,2,0


In [ ]:
# Step 1: Count reviews by airline and region
region_counts = df_filtered_na.groupby(['airline_name', 'author_region']).size().unstack(fill_value=0)

# Step 2: Convert counts to row-wise percentages
region_pct = region_counts.div(region_counts.sum(axis=1), axis=0) * 100

# Step 3: Round and display
region_pct = region_pct.round(1)
region_pct


author_region,Africa,Asia,Europe,North America,Oceania,South America,Unknown
airline_name,,,,,,,
air-canada,0.0,3.9,19.1,71.9,4.7,0.2,0.2
alaska-airlines,0.0,2.1,5.5,81.4,10.2,0.8,0.0
american-airlines,0.5,2.8,26.1,65.2,2.9,1.8,0.7
delta-air-lines,0.6,4.3,18.9,69.3,5.2,1.7,0.0
jetblue-airways,1.3,3.8,12.2,79.7,2.1,0.8,0.0
southwest-airlines,0.5,1.1,5.5,86.0,5.5,1.3,0.0
united-airlines,0.4,5.4,15.4,72.9,5.0,1.0,0.1


In [ ]:
# Step 1: Count reviews by airline and region
region_counts = df_filtered_na.groupby(['airline_name', 'author_region']).size().unstack(fill_value=0)

# Step 2: Convert counts to column-wise percentages
region_pct_col = region_counts.div(region_counts.sum(axis=0), axis=1) * 100

# Step 3: Round and display
region_pct_col = region_pct_col.round(1)
region_pct_col


author_region,Africa,Asia,Europe,North America,Oceania,South America,Unknown
airline_name,,,,,,,
air-canada,0.0,16.0,17.6,14.7,14.6,2.7,16.7
alaska-airlines,0.0,4.2,2.4,8.0,15.3,5.4,0.0
american-airlines,21.4,14.3,30.0,16.7,11.5,29.7,66.7
delta-air-lines,21.4,16.8,16.5,13.5,15.3,21.6,0.0
jetblue-airways,21.4,7.6,5.4,7.9,3.2,5.4,0.0
southwest-airlines,14.3,3.4,3.9,13.6,13.4,13.5,0.0
united-airlines,21.4,37.8,24.2,25.6,26.8,21.6,16.7
